<a href="https://colab.research.google.com/github/israval08/Parking-Cat/blob/main/Machinelearning-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Machine learnig.
Israel Valenzuela

1.- Tomar un modelo de AI y aplicar fine-tunning (“tunearlo”), esto es “el proceso de adaptar un modelo previamente entrenado para tareas o casos de uso específicos. … es una técnica fundamental de Deep learning, especialmente en el proceso de entrenamiento de modelos fundacionales utilizados para la IA generativa” (Bergmann. D “El fine tunning en machine learning”, en [ibm.com/es-es/topics/fine-tuning])

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from datasets import load_dataset

# Cargar el dataset IMDB
dataset = load_dataset("imdb")

# Reducir el dataset
train_dataset = dataset["train"].select(range(500))  # 500 ejemplos para entrenamiento
val_dataset = dataset["test"].select(range(100))    # 100 ejemplos para validación


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Cargar el tokenizador y el modelo
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",                 # Carpeta de salida
    evaluation_strategy="steps",           # Evaluar por pasos
    save_strategy="steps",                 # Guardar por pasos
    save_steps=50,                         # Guardar cada 50 pasos
    logging_steps=10,                      # Registrar logs cada 10 pasos
    learning_rate=2e-5,                    # Tasa de aprendizaje
    per_device_train_batch_size=8,         # Reducir el tamaño del lote
    num_train_epochs=1,                    # Reducir a 1 época
    weight_decay=0.01,                     # Decaimiento de pesos
    save_total_limit=2,                    # Limitar modelos guardados
    load_best_model_at_end=True,           # Cargar el mejor modelo al final
    report_to="none"                       # Desactivar WandB
)


In [ ]:
from transformers import Trainer

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Change 'label' key to 'labels'
train_dataset = train_dataset.rename_column("label", "labels")
val_dataset = val_dataset.rename_column("label", "labels")

# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Entrenar el modelo
trainer.train()

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
10,0.425900,0.164969
20,0.091800,0.035126
30,0.026200,0.014215
40,0.012400,0.008966
50,0.009100,0.007157
60,0.007900,0.006561


Could not locate the best model at ./results/checkpoint-60/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=63, training_loss=0.09138902903549255, metrics={'train_runtime': 2394.1559, 'train_samples_per_second': 0.209, 'train_steps_per_second': 0.026, 'total_flos': 66233699328000.0, 'train_loss': 0.09138902903549255, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./modelo_entrenado")
tokenizer.save_pretrained("./modelo_entrenado")


('./modelo_entrenado/tokenizer_config.json',
 './modelo_entrenado/special_tokens_map.json',
 './modelo_entrenado/vocab.txt',
 './modelo_entrenado/added_tokens.json',
 './modelo_entrenado/tokenizer.json')

2.- Utilizando alguno de estos sitios https://roboflow.com/ o https://huggingface.co/, seleccionar dos (2) modelos simples, las librerías (compatibles con colab o jupyter), datos etc.

In [ ]:
!pip install transformers
!pip install torch



In [ ]:
!pip install datasets

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Cargar el tokenizador y el modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Texto de ejemplo
text = "Hugging Face is creating a tool that democratizes AI."

# Tokenizar el texto
inputs = tokenizer(text, return_tensors='pt')

# Obtener las salidas del modelo
with torch.no_grad():
    outputs = model(**inputs)

# Mostrar las salidas
print(outputs)


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1825,  0.0843,  0.0436,  ..., -0.6633, -0.0026,  0.5883],
         [ 0.3064, -0.5028,  0.9039,  ..., -0.3088,  0.6013,  0.4261],
         [ 0.2644, -0.4869,  0.6634,  ..., -0.6048, -0.0349,  0.1243],
         ...,
         [-0.0312,  0.1046,  0.5757,  ..., -0.9736, -0.4921, -0.2229],
         [ 0.4069,  0.2166, -0.2082,  ..., -0.1257, -0.6284, -0.4286],
         [-0.5190,  0.2051,  0.2240,  ..., -0.3013, -0.7027, -0.2072]]]), pooler_output=tensor([[-8.6306e-01, -4.8507e-01, -8.5299e-01,  7.1729e-01,  6.1560e-01,
         -3.1037e-01,  5.5880e-01,  3.0846e-01, -6.7352e-01, -9.9993e-01,
         -3.4989e-01,  7.7340e-01,  9.7119e-01,  3.5364e-01,  8.5608e-01,
         -5.0894e-01, -4.8986e-02, -4.8850e-01,  3.8780e-01,  1.8295e-01,
          5.7000e-01,  9.9996e-01,  6.6720e-06,  3.6832e-01,  5.1577e-01,
          9.3869e-01, -7.5014e-01,  8.8402e-01,  9.2303e-01,  7.1994e-01,
         -4.7790e-01,  3.2389e-01, -

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Cargar el tokenizador y el modelo DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Texto de ejemplo
text = "I love using Hugging Face models!"

# Tokenizar el texto
inputs = tokenizer(text, return_tensors='pt')

# Obtener las salidas del modelo
with torch.no_grad():
    outputs = model(**inputs)

# Mostrar las salidas
print(outputs)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SequenceClassifierOutput(loss=None, logits=tensor([[-0.0921, -0.0149]]), hidden_states=None, attentions=None)


In [ ]:
from datasets import load_dataset

# Cargar el conjunto de datos IMDb
dataset = load_dataset('imdb')

# Preparar los datos para el entrenamiento
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_datasets = dataset.map(preprocess_function, batched=True)


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Cargar el tokenizador y el modelo DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset
from transformers import DistilBertTokenizer
import time

# Cargar el conjunto de datos IMDb
dataset = load_dataset('imdb')

# Seleccionar el 5% de los datos
def select_subset(dataset, fraction=0.05):
    num_samples = int(len(dataset) * fraction)
    return dataset.select(range(num_samples))

# Aplicar la selección a los conjuntos de entrenamiento y prueba
train_dataset = select_subset(dataset['train'])
test_dataset = select_subset(dataset['test'])

# Cargar el tokenizador
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Definir la función de preprocesamiento
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

# Aplicar la función de preprocesamiento a los datos
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['text'])
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=['text'])


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertForSequenceClassification

# Cargar el modelo
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",  # Cambiado de evaluation_strategy a eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=[]  # Deshabilitar wandb si no lo estás usando
)

# Configurar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

# Entrenar el modelo
start_time = time.time()
trainer.train()
end_time = time.time()

# Calcular el tiempo total de entrenamiento
total_time = end_time - start_time
print(f"Tiempo total de entrenamiento: {total_time / 60:.2f} minutos")


Epoch,Training Loss,Validation Loss
1,No log,0.001878
2,No log,0.000911


Epoch,Training Loss,Validation Loss
1,No log,0.001878
2,No log,0.000911
3,No log,0.000744


Tiempo total de entrenamiento: 288.68 minutos


In [ ]:
# Evaluar el modelo
results = trainer.evaluate()
print(results)


{'eval_loss': 0.0007440066547133029, 'eval_runtime': 1233.1481, 'eval_samples_per_second': 1.014, 'eval_steps_per_second': 0.064, 'epoch': 3.0}


Análisis de los resultados de la evaluación
eval_loss: 0.000744

La pérdida de evaluación (eval_loss) es un indicador de cuán bien el modelo está generalizando al conjunto de datos de validación. En este caso, la pérdida es muy baja, lo que sugiere que el modelo se ha entrenado bien.
eval_runtime: 1233.1481 segundos (aproximadamente 20.55 minutos)

Este es el tiempo que tomó la evaluación.
eval_samples_per_second: 1.014

La velocidad a la que se procesaron los ejemplos durante la evaluación.
eval_steps_per_second: 0.064

La velocidad a la que se procesaron los pasos (batches) durante la evaluación.
epoch: 3.0

Indica que la evaluación se realizó después de la tercera época de entrenamiento.

3.- Personalizar el modelo obtenido (pipeline automatizado), en el entorno de desarrollo código (IDE) y documentar los pasos realizados, en este mismo archivo. Realizar la tarea en Google colab y jupyter (.ipynb).

In [ ]:
from transformers import pipeline

# Crear un pipeline de clasificación de texto
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Realizar una predicción
result = classifier("I love using Hugging Face models!")
print(result)


[{'label': 'LABEL_0', 'score': 0.9926002025604248}]


Análisis del resultado
label: 'LABEL_0': Esto indica la clase a la que el modelo ha asignado el texto de entrada. En el conjunto de datos IMDb, LABEL_0 generalmente corresponde a una reseña negativa, y LABEL_1 corresponde a una reseña positiva.
score: 0.9926002025604248: Esto es la probabilidad o confianza con la que el modelo ha asignado la clase LABEL_0. En este caso, el modelo está muy seguro (99.26%) de que el texto de entrada pertenece a la clase LABEL_0.
Interpretación
Dado que el texto de entrada era "I love using Hugging Face models!", que es claramente una declaración positiva, parece que el modelo ha cometido un error en esta predicción. Esto puede deberse a varios factores, como la calidad del entrenamiento, el tamaño del conjunto de datos, o la configuración del modelo.

In [ ]:
# Guardar el modelo y el tokenizador
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json')